In [6]:
import numpy as np
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel

In [34]:
dfbrew = pd.read_csv("../raw_data/Beers_Breweries_and_Beer Reviews/breweries.csv")
dfbeer = pd.read_csv("../raw_data/beers_style_renamed.csv")
dftop = pd.read_csv("../raw_data/top_beer_info_style_renamed.csv")

corres_xls = pd.read_excel(
            '../assets/correspondance_breweryclean2.xlsx')

corres_xls.drop(columns='Unnamed: 0', inplace=True)

corres_xls.set_index(1, inplace=True)

corres= corres_xls.to_dict()

corres[0]

dfbrew = dfbrew.rename(columns={"name": "brewery"})
dfbrew = dfbrew.rename(columns={"id": "brewery_id"})

#Replace brewery via corres
dfbrew['brewery'].replace(corres[0], inplace=True)


dfbrewb =  pd.merge(dfbeer,dfbrew[['brewery_id','brewery']],how='left',on=['brewery_id'])

dftopbrew = pd.merge(dftop,dfbrewb,how='inner',on=['name','brewery'])






In [35]:
dftopbrew = dftopbrew[dftopbrew['retired'] == "f"]
dftopbrew

,name,key,style_x,style key,brewery,description,abv_x,ave rating,min ibu,max ibu,...,malty,id,brewery_id,state,country,style_y,availability,abv_y,notes,retired
0,Double Bag,252,Altbier,8,Long Trail Brewing Co.,"Notes:This malty, full-bodied double alt is al...",7.2,3.90,25,50,...,84,273,94,VT,US,Altbier - German,Year-round,7.2,"This malty, full-bodied double alt is also kno...",f
1,Long Trail Ale,253,Altbier,8,Long Trail Brewing Co.,Notes:Long Trail Ale is a full-bodied amber al...,5.0,3.58,25,50,...,62,3128,94,VT,US,Altbier - German,Year-round,5.0,Long Trail Ale is a full-bodied amber ale mode...,f
2,Scurry,255,Altbier,8,Off Color Brewing,Notes:Just cause it's dark and German doesn't ...,5.3,3.67,25,50,...,218,94505,31678,IL,US,Altbier - German,Year-round,5.3,Just cause it's dark and German doesn't mean i...,f
3,Sleigh'r Dark Doüble Alt Ale,256,Altbier,8,Ninkasi Brewing Company,Notes:Called 'Dark Double Alt' on the label.Se...,7.2,3.78,25,50,...,95,47120,14400,OR,US,Altbier - German,Winter,7.2,Called 'Dark Double Alt' on the label.,f
4,Okto Festival Ale,258,Altbier,8,Widmer Brothers Brewing Company,Notes:Classified as a German Altbier by the Wi...,5.3,3.46,25,50,...,97,25752,8,OR,US,Altbier - German,Fall,5.3,Classified as a German Altbier by the Widmer B...,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4389,The First Snow Ale,695,Winter Warmer,17,RJ Rockers Brewing Company,Notes:This hearty American pale ale contains a...,6.0,3.63,35,50,...,58,26344,4408,SC,US,Winter Warmer,Fall,6.0,This hearty American pale ale contains a rich ...,f
4390,Red Nose Winter Ale,696,Winter Warmer,17,Natty Greene's Pub & Brewing Co.,Notes:,6.8,3.59,35,50,...,77,21003,9792,NC,US,Winter Warmer,Winter,6.8,No notes at this time.,f
4391,Fish Tale Winterfish,698,Winter Warmer,17,Fish Brewing Company / Fishbowl Brewpub,Notes:,7.5,3.76,35,50,...,73,3579,684,WA,US,Winter Warmer,Winter,7.5,No notes at this time.,f
4392,"He'Brew Hanukkah, Chanukah: Pass The Beer",699,Winter Warmer,17,Shmaltz Brewing Company,Notes:Chanukah Beer pours a rich crystal clear...,8.0,3.61,35,50,...,129,145097,262,NY,US,Winter Warmer,Rotating,8.0,Chanukah Beer pours a rich crystal clear chest...,f


In [37]:
dftopbrew= dftopbrew[["name","style_x","abv_x","ave rating",'min ibu', 'max ibu', 'astringency', 'body',
                      
       'alcohol', 'bitter', 'sweet', 'sour', 'salty', 'fruits', 'hoppy',
       'spices', 'malty']]

                     
dftopbrew 




,name,style_x,abv_x,ave rating,min ibu,max ibu,astringency,body,alcohol,bitter,sweet,sour,salty,fruits,hoppy,spices,malty
0,Double Bag,Altbier,7.2,3.90,25,50,12,57,18,33,55,16,0,24,35,12,84
1,Long Trail Ale,Altbier,5.0,3.58,25,50,14,37,6,42,43,11,0,10,54,4,62
2,Scurry,Altbier,5.3,3.67,25,50,21,69,10,63,120,14,0,19,36,15,218
3,Sleigh'r Dark Doüble Alt Ale,Altbier,7.2,3.78,25,50,25,51,26,44,45,9,1,11,51,20,95
4,Okto Festival Ale,Altbier,5.3,3.46,25,50,28,40,3,40,58,29,0,36,54,8,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4389,The First Snow Ale,Winter Warmer,6.0,3.63,35,50,15,31,23,16,54,43,0,54,14,140,58
4390,Red Nose Winter Ale,Winter Warmer,6.8,3.59,35,50,8,44,24,19,52,21,0,26,21,96,77
4391,Fish Tale Winterfish,Winter Warmer,7.5,3.76,35,50,11,36,50,70,72,59,0,81,110,18,73
4392,"He'Brew Hanukkah, Chanukah: Pass The Beer",Winter Warmer,8.0,3.61,35,50,6,64,30,57,78,15,1,28,57,23,129


In [10]:
 df['overview'][0]
'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

    # Creating an object of Tfidf vector class and fitting it on the overview of movies
    tfidf = TfidfVectorizer(stop_words='english',analyzer='word',min_df=3,strip_accents='unicode'
                         ,ngram_range=(1,3),token_pattern=r'\w{1,}',max_features=None)

    df['overview'].fillna('',inplace=True)
    vector = tfidf.fit_transform(df['overview'])
    vector


Index(['name', 'key', 'style_x', 'style key', 'brewery', 'description',
       'abv_x', 'ave rating', 'min ibu', 'max ibu', 'astringency', 'body',
       'alcohol', 'bitter', 'sweet', 'sour', 'salty', 'fruits', 'hoppy',
       'spices', 'malty', 'id', 'brewery_id', 'state', 'country', 'style_y',
       'availability', 'abv_y', 'notes', 'retired'],
      dtype='object')